In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sc
import math
import datetime
from sklearn.feature_selection import SelectKBest, mutual_info_regression, RFE
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
from sklearn import tree

In [4]:
training_df = pd.read_csv("flights_train.csv")

In [5]:
training_y = training_df['ARR_DELAY']

In [6]:
day_of_year = [datetime.datetime.strptime(date_str, '%Y-%m-%d').timetuple().tm_yday for date_str in training_df['FL_DATE'] ]
month = [datetime.datetime.strptime(date_str, '%Y-%m-%d').timetuple().tm_mon for date_str in training_df['FL_DATE'] ]
training_x = training_df.drop(['UID','FL_NUM','AIRLINE_ID','ORIGIN_CITY_NAME','DISTANCE', 'FL_DATE', 'DEST_CITY_NAME','ORIGIN_STATE_ABR','DEST_STATE_ABR','ARR_DELAY', ], 
                 axis=1)  
training_x['month'] = month
training_x['day_of_year'] = day_of_year

d = {'ORIGIN': training_x['ORIGIN']}
df = pd.DataFrame(data=d)
e = {'FIRST_DEP_TIME': training_x['FIRST_DEP_TIME']}
ef = pd.DataFrame(data=e)
f = {'UNIQUE_CARRIER': training_x['UNIQUE_CARRIER']}
ff = pd.DataFrame(data=f)
g = {'DEST': training_x['DEST']}
gf = pd.DataFrame(data=g)

print(training_x.shape)
first_dep_ohe = pd.get_dummies(ef['FIRST_DEP_TIME'])
print('first dep shape: '+str(first_dep_ohe.shape))
training_x = pd.concat([training_x, first_dep_ohe], axis=1)
print(training_x.shape)
training_x = training_x.drop(['FIRST_DEP_TIME', ], axis=1)
print(training_x.shape)
origin_ohe = pd.get_dummies(df['ORIGIN'])
print('origin shape: '+str(origin_ohe.shape))
training_x = pd.concat([training_x, origin_ohe], axis=1)
print(training_x.shape)
training_x = training_x.drop(['ORIGIN', ], axis=1)
print(training_x.shape)
dest_ohe = pd.get_dummies(gf['DEST'])
print(dest_ohe.columns[0:4])
dest_ohe.rename(columns=lambda x: x+'_DEST', inplace=True)
print(dest_ohe.columns[0:4])
print('dest shape: '+str(dest_ohe.shape))
training_x = pd.concat([training_x, dest_ohe], axis=1)
print(training_x.shape)
training_x = training_x.drop(['DEST', ], axis=1)
print(training_x.shape)
carrier_ohe = pd.get_dummies(ff['UNIQUE_CARRIER'])
print('carrier shape: '+str(carrier_ohe.shape))
training_x = pd.concat([training_x, carrier_ohe], axis=1)
print(training_x.shape)
training_x = training_x.drop(['UNIQUE_CARRIER', ], axis=1)
print(training_x.shape)
training_x.info()

(4911, 14)
first dep shape: (4911, 29)
(4911, 43)
(4911, 42)
origin shape: (4911, 239)
(4911, 281)
(4911, 280)
Index(['ABE', 'ABQ', 'ABR', 'ACY'], dtype='object')
Index(['ABE_DEST', 'ABQ_DEST', 'ABR_DEST', 'ACY_DEST'], dtype='object')
dest shape: (4911, 228)
(4911, 508)
(4911, 507)
carrier shape: (4911, 12)
(4911, 519)
(4911, 518)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4911 entries, 0 to 4910
Columns: 518 entries, DAY_OF_WEEK to WN
dtypes: int64(10), uint8(508)
memory usage: 2.8 MB


In [7]:
training_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4911 entries, 0 to 4910
Columns: 518 entries, DAY_OF_WEEK to WN
dtypes: int64(10), uint8(508)
memory usage: 2.8 MB


In [9]:
print(training_x.shape)
x_test = training_x
print(training_y.shape)
X_new = SelectKBest(mutual_info_regression, k=3).fit_transform(x_test, training_y)
X_new[0:5]

(4911, 518)
(4911,)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


array([[ 11, 373,   0],
       [ 13, 183,   0],
       [ 48, 355,   0],
       [ 17, 136,   0],
       [ 37, 114,   0]], dtype=int64)

In [10]:
x_test[0:5]

,DAY_OF_WEEK,CRS_DEP_TIME,TAXI_OUT,TAXI_IN,ACTUAL_ELAPSED_TIME,DISTANCE_GROUP,ORIGIN_CITY_MARKET_ID,DEST_CITY_MARKET_ID,month,day_of_year,...,B6,DL,EV,F9,HA,NK,OO,UA,VX,WN
0,2,1020,11,8,373,11,30559,32982,1,3,...,0,0,0,0,0,0,0,0,0,0
1,6,1220,13,9,183,5,33570,30194,1,28,...,0,0,0,0,0,0,0,0,0,0
2,6,945,48,6,355,11,30299,30466,1,21,...,0,0,0,0,0,0,0,0,0,0
3,1,1437,17,8,136,4,32457,30325,1,23,...,0,0,0,0,0,0,0,1,0,0
4,1,830,37,7,114,3,30397,30852,1,2,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
# takes a long time
rfe = RFE(SVR(kernel="linear"), 4)
fit = rfe.fit(x_test, training_y)

In [25]:
fit.ranking_

array([1, 5, 2, 1, 4, 1, 6, 7, 1, 3])

In [37]:
clf = tree.DecisionTreeClassifier(max_depth=3)
clf = clf.fit(training_x, training_y)


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
epochs = 10
for e in range(epochs):
    model.fit(training_x, training_y)
    print(model.score(validation_x, validation_y))

model.predict(test_x)

